In [3]:
import os
import joblib
import json
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import (
    classification_report, confusion_matrix,
    f1_score, accuracy_score, precision_score, recall_score
)
import shap
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

OUTPUT_DIR = r"F:\Ai&ml\outputs"
MODEL_DIR = os.path.join(OUTPUT_DIR, "models")
RESULTS_DIR = os.path.join(OUTPUT_DIR, "results")
VIZ_DIR = os.path.join(OUTPUT_DIR, "visualization")
DATASET_DIR = os.path.join(OUTPUT_DIR, "datasets")
SHAP_DIR = os.path.join(VIZ_DIR, "shap_analysis")

os.makedirs(SHAP_DIR, exist_ok=True)

print("="*70)
print("LIVER CIRRHOSIS MODEL EVALUATION & EXPLAINABILITY")
print("="*70)
print(f"\nOutput directories:")
print(f"  • Models: {MODEL_DIR}")
print(f"  • Results: {RESULTS_DIR}")
print(f"  • Visualizations: {VIZ_DIR}")
print(f"  • SHAP Analysis: {SHAP_DIR}")

print("\n" + "="*70)
print("LOADING DATA AND MODELS")
print("="*70)

X_test = joblib.load(os.path.join(DATASET_DIR, 'X_test.joblib'))
y_test = joblib.load(os.path.join(DATASET_DIR, 'y_test.joblib'))
feature_names = joblib.load(os.path.join(DATASET_DIR, 'feature_names.joblib'))

label_encoder = joblib.load(os.path.join(DATASET_DIR, 'label_encoder.joblib'))
class_names = [str(cls) for cls in label_encoder.classes_]

try:
    initial_model = joblib.load(os.path.join(MODEL_DIR, 'best_initial_model.joblib'))
    initial_model_name = joblib.load(os.path.join(MODEL_DIR, 'best_model_name.joblib'))
    print(f"\n✓ Initial model loaded: {initial_model_name}")
except:
    initial_model = None
    print("\n⚠️  Initial model not found")

try:
    tuned_model = joblib.load(os.path.join(MODEL_DIR, 'best_tuned_model.joblib'))
    print("✓ Tuned model loaded")
except:
    tuned_model = None
    print("⚠️  Tuned model not found")

print(f"\n✓ Test data loaded:")
print(f"  • Samples: {len(X_test)}")
print(f"  • Features: {len(feature_names)}")
print(f"  • Classes: {class_names}")

def evaluate_model_comprehensive(model, X_test, y_test, model_name,
                                 class_names, save_prefix="model"):

    print(f"\n{'='*70}")
    print(f"EVALUATING: {model_name}")
    print(f"{'='*70}")

    y_pred = model.predict(X_test)

    accuracy = accuracy_score(y_test, y_pred)
    weighted_f1 = f1_score(y_test, y_pred, average='weighted')
    macro_f1 = f1_score(y_test, y_pred, average='macro')
    weighted_precision = precision_score(y_test, y_pred, average='weighted', zero_division=0)
    weighted_recall = recall_score(y_test, y_pred, average='weighted', zero_division=0)

    print(f"\n📊 Overall Metrics:")
    print(f"  • Accuracy: {accuracy:.4f}")
    print(f"  • Weighted F1: {weighted_f1:.4f}")
    print(f"  • Macro F1: {macro_f1:.4f}")
    print(f"  • Weighted Precision: {weighted_precision:.4f}")
    print(f"  • Weighted Recall: {weighted_recall:.4f}")

    report = classification_report(y_test, y_pred,
                                   target_names=class_names,
                                   output_dict=True,
                                   zero_division=0)
    report_df = pd.DataFrame(report).transpose()

    print(f"\n📋 Classification Report:")
    print(report_df.round(4))

    report_df.to_csv(
        os.path.join(RESULTS_DIR, f'{save_prefix}_classification_report.csv')
    )

    cm = confusion_matrix(y_test, y_pred)

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names,
                square=True, linewidths=1, linecolor='black',
                cbar_kws={'label': 'Count'})
    plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
    plt.ylabel('True Label', fontsize=12, fontweight='bold')
    plt.title(f'Confusion Matrix - {model_name}',
              fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig(os.path.join(VIZ_DIR, f'{save_prefix}_confusion_matrix.png'),
                dpi=300, bbox_inches='tight')
    plt.close()
    print(f"\n✓ Confusion matrix saved")

    cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

    plt.figure(figsize=(10, 8))
    sns.heatmap(cm_normalized, annot=True, fmt='.2%', cmap='RdYlGn',
                xticklabels=class_names,
                yticklabels=class_names,
                square=True, linewidths=1, linecolor='black',
                cbar_kws={'label': 'Percentage'})
    plt.xlabel('Predicted Label', fontsize=12, fontweight='bold')
    plt.ylabel('True Label', fontsize=12, fontweight='bold')
    plt.title(f'Normalized Confusion Matrix - {model_name}',
              fontsize=16, fontweight='bold', pad=20)
    plt.tight_layout()
    plt.savefig(os.path.join(VIZ_DIR, f'{save_prefix}_confusion_matrix_normalized.png'),
                dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Normalized confusion matrix saved")

    per_class_metrics = {
        'Class': class_names,
        'Precision': [report[cls]['precision'] for cls in class_names],
        'Recall': [report[cls]['recall'] for cls in class_names],
        'F1-Score': [report[cls]['f1-score'] for cls in class_names],
        'Support': [report[cls]['support'] for cls in class_names]
    }

    metrics_df = pd.DataFrame(per_class_metrics)

    fig, axes = plt.subplots(1, 3, figsize=(18, 5))

    for idx, metric in enumerate(['Precision', 'Recall', 'F1-Score']):
        ax = axes[idx]
        bars = ax.bar(metrics_df['Class'], metrics_df[metric],
                     color='skyblue', edgecolor='black', linewidth=1.5)

        for bar in bars:
            height = bar.get_height()
            ax.text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.3f}', ha='center', va='bottom', fontweight='bold')

        ax.set_xlabel('Class', fontsize=11, fontweight='bold')
        ax.set_ylabel(metric, fontsize=11, fontweight='bold')
        ax.set_title(f'{metric} by Class', fontsize=13, fontweight='bold')
        ax.set_ylim(0, 1.1)
        ax.grid(axis='y', alpha=0.3, linestyle='--')

    plt.suptitle(f'Per-Class Metrics - {model_name}',
                fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig(os.path.join(VIZ_DIR, f'{save_prefix}_per_class_metrics.png'),
                dpi=300, bbox_inches='tight')
    plt.close()
    print(f"✓ Per-class metrics visualization saved")

    evaluation_results = {
        'model_name': model_name,
        'timestamp': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
        'overall_metrics': {
            'accuracy': float(accuracy),
            'weighted_f1': float(weighted_f1),
            'macro_f1': float(macro_f1),
            'weighted_precision': float(weighted_precision),
            'weighted_recall': float(weighted_recall)
        },
        'confusion_matrix': cm.tolist(),
        'confusion_matrix_normalized': cm_normalized.tolist(),
        'classification_report': report,
        'per_class_metrics': metrics_df.to_dict('records')
    }

    with open(os.path.join(RESULTS_DIR, f'{save_prefix}_evaluation_results.json'), 'w') as f:
        json.dump(evaluation_results, f, indent=4)

    print(f"✓ Evaluation results saved to JSON")

    return y_pred, report_df, evaluation_results

def explain_model_with_shap(model, X_test, feature_names, model_name,
                           sample_size=100, save_prefix="model"):

    print(f"\n{'='*70}")
    print(f"SHAP ANALYSIS: {model_name}")
    print(f"{'='*70}")

    if hasattr(model, 'named_steps'):
        actual_model = model.named_steps['model']
    else:
        actual_model = model

    X_sample = X_test.iloc[:sample_size] if hasattr(X_test, 'iloc') else X_test[:sample_size]

    print(f"\n⏳ Computing SHAP values for {sample_size} samples...")

    try:
        explainer = shap.TreeExplainer(actual_model)
        shap_values = explainer.shap_values(X_sample)

        if isinstance(shap_values, list):
            print(f"✓ Multi-class SHAP values computed ({len(shap_values)} classes)")
        else:
            print(f"✓ SHAP values computed")

        print("\n📊 Generating SHAP summary plots...")
        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X_sample,
                         feature_names=feature_names,
                         plot_type="bar", show=False)
        plt.title(f'SHAP Feature Importance - {model_name}',
                 fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.savefig(os.path.join(SHAP_DIR, f'{save_prefix}_shap_importance.png'),
                   dpi=300, bbox_inches='tight')
        plt.close()
        print("  ✓ Feature importance plot saved")

        plt.figure(figsize=(12, 8))
        shap.summary_plot(shap_values, X_sample,
                         feature_names=feature_names, show=False)
        plt.title(f'SHAP Feature Effects - {model_name}',
                 fontsize=16, fontweight='bold', pad=20)
        plt.tight_layout()
        plt.savefig(os.path.join(SHAP_DIR, f'{save_prefix}_shap_summary.png'),
                   dpi=300, bbox_inches='tight')
        plt.close()
        print("  ✓ Feature effects plot saved")

        if isinstance(shap_values, list):
            mean_shap = np.abs(np.array(shap_values)).mean(axis=0).mean(axis=0).flatten()
        else:
            mean_shap = np.abs(shap_values).mean(axis=0).flatten()

        if isinstance(feature_names, np.ndarray):
            feature_names_flat = feature_names.flatten().tolist()
        elif isinstance(feature_names, pd.Index):
            feature_names_flat = feature_names.tolist()
        else:
            feature_names_flat = list(feature_names)

        if len(mean_shap) != len(feature_names_flat):
            print(f"  ⚠️  Warning: SHAP values length ({len(mean_shap)}) doesn't match feature names ({len(feature_names_flat)})")
            min_len = min(len(mean_shap), len(feature_names_flat))
            mean_shap = mean_shap[:min_len]
            feature_names_flat = feature_names_flat[:min_len]

        feature_importance_df = pd.DataFrame({
            'feature': feature_names_flat,
            'mean_abs_shap': mean_shap
        }).sort_values('mean_abs_shap', ascending=False)

        feature_importance_df.to_csv(
            os.path.join(SHAP_DIR, f'{save_prefix}_shap_feature_importance.csv'),
            index=False
        )

        print(f"\n🏆 Top 10 Most Important Features (by SHAP):")
        print(feature_importance_df.head(10).to_string(index=False))

        print("\n📈 Generating dependence plots for top 5 features...")
        top_features = feature_importance_df.head(5)['feature'].tolist()

        for i, feature in enumerate(top_features):
            try:
                feature_idx = feature_names.index(feature)
                plt.figure(figsize=(10, 6))

                if isinstance(shap_values, list):
                    shap.dependence_plot(feature_idx, shap_values[0], X_sample,
                                       feature_names=feature_names, show=False)
                else:
                    shap.dependence_plot(feature_idx, shap_values, X_sample,
                                       feature_names=feature_names, show=False)

                plt.title(f'SHAP Dependence Plot - {feature}',
                         fontsize=14, fontweight='bold')
                plt.tight_layout()
                plt.savefig(os.path.join(SHAP_DIR,
                           f'{save_prefix}_dependence_{i+1}_{feature}.png'),
                           dpi=300, bbox_inches='tight')
                plt.close()
                print(f"  ✓ Dependence plot saved for {feature}")
            except Exception as e:
                print(f"  ⚠️  Could not create dependence plot for {feature}: {str(e)}")

        print("\n🔍 Generating force plots for sample predictions...")
        for i in range(min(5, len(X_sample))):
            try:
                plt.figure(figsize=(20, 3))

                if isinstance(shap_values, list):
                    class_idx = np.argmax([sv[i].sum() for sv in shap_values])
                    shap.force_plot(explainer.expected_value[class_idx],
                                  shap_values[class_idx][i],
                                  X_sample.iloc[i] if hasattr(X_sample, 'iloc') else X_sample[i],
                                  feature_names=feature_names,
                                  matplotlib=True, show=False)
                else:
                    shap.force_plot(explainer.expected_value,
                                  shap_values[i],
                                  X_sample.iloc[i] if hasattr(X_sample, 'iloc') else X_sample[i],
                                  feature_names=feature_names,
                                  matplotlib=True, show=False)

                plt.title(f'SHAP Force Plot - Sample {i+1}',
                         fontsize=12, fontweight='bold')
                plt.tight_layout()
                plt.savefig(os.path.join(SHAP_DIR,
                           f'{save_prefix}_force_plot_sample_{i+1}.png'),
                           dpi=300, bbox_inches='tight')
                plt.close()
            except Exception as e:
                print(f"  ⚠️  Could not create force plot for sample {i+1}: {str(e)}")

        print(f"\n✅ SHAP analysis completed successfully!")

        return shap_values, feature_importance_df

    except Exception as e:
        print(f"\n❌ Error during SHAP analysis: {str(e)}")
        print("This might occur if the model type is not supported by TreeExplainer.")
        return None, None

def compare_models(results_dict):

    print(f"\n{'='*70}")
    print("MODEL COMPARISON")
    print(f"{'='*70}")

    comparison_df = pd.DataFrame({
        'Model': list(results_dict.keys()),
        'Accuracy': [r['overall_metrics']['accuracy'] for r in results_dict.values()],
        'Weighted F1': [r['overall_metrics']['weighted_f1'] for r in results_dict.values()],
        'Macro F1': [r['overall_metrics']['macro_f1'] for r in results_dict.values()],
        'Precision': [r['overall_metrics']['weighted_precision'] for r in results_dict.values()],
        'Recall': [r['overall_metrics']['weighted_recall'] for r in results_dict.values()]
    })

    print("\n📊 Model Comparison:")
    print(comparison_df.round(4).to_string(index=False))

    comparison_df.to_csv(
        os.path.join(RESULTS_DIR, 'model_comparison_detailed.csv'),
        index=False
    )

    metrics = ['Accuracy', 'Weighted F1', 'Macro F1', 'Precision', 'Recall']

    fig, ax = plt.subplots(figsize=(12, 6))
    x = np.arange(len(comparison_df))
    width = 0.15

    for i, metric in enumerate(metrics):
        ax.bar(x + i*width, comparison_df[metric], width,
               label=metric, alpha=0.8)

    ax.set_xlabel('Model', fontsize=12, fontweight='bold')
    ax.set_ylabel('Score', fontsize=12, fontweight='bold')
    ax.set_title('Model Performance Comparison', fontsize=16, fontweight='bold')
    ax.set_xticks(x + width * 2)
    ax.set_xticklabels(comparison_df['Model'])
    ax.legend()
    ax.grid(axis='y', alpha=0.3, linestyle='--')
    ax.set_ylim(0, 1.1)

    plt.tight_layout()
    plt.savefig(os.path.join(VIZ_DIR, 'model_comparison_all_metrics.png'),
                dpi=300, bbox_inches='tight')
    plt.close()

    print(f"\n✓ Comparison visualization saved")

    return comparison_df

if __name__ == "__main__":
    start_time = datetime.now()

    results_dict = {}

    if initial_model is not None:
        y_pred_initial, report_initial, results_initial = evaluate_model_comprehensive(
            initial_model, X_test, y_test,
            initial_model_name, class_names,
            save_prefix="initial_model"
        )
        results_dict[f'{initial_model_name} (Initial)'] = results_initial

        shap_values_initial, shap_importance_initial = explain_model_with_shap(
            initial_model, X_test, feature_names,
            initial_model_name,
            sample_size=100,
            save_prefix="initial_model"
        )

    if tuned_model is not None:
        y_pred_tuned, report_tuned, results_tuned = evaluate_model_comprehensive(
            tuned_model, X_test, y_test,
            "Tuned Model", class_names,
            save_prefix="tuned_model"
        )
        results_dict['Tuned Model'] = results_tuned

        shap_values_tuned, shap_importance_tuned = explain_model_with_shap(
            tuned_model, X_test, feature_names,
            "Tuned Model",
            sample_size=100,
            save_prefix="tuned_model"
        )

    if len(results_dict) > 1:
        comparison_df = compare_models(results_dict)

    end_time = datetime.now()
    duration = (end_time - start_time).total_seconds()

    print("\n" + "="*70)
    print("✅ EVALUATION & EXPLAINABILITY COMPLETED!")
    print("="*70)
    print(f"\n⏱️  Total execution time: {duration:.2f} seconds")
    print(f"\n📁 All outputs saved to:")
    print(f"  • Results: {RESULTS_DIR}")
    print(f"  • Visualizations: {VIZ_DIR}")
    print(f"  • SHAP Analysis: {SHAP_DIR}")
    print("\n" + "="*70)
    print("By_OwenXAGK")

LIVER CIRRHOSIS MODEL EVALUATION & EXPLAINABILITY

Output directories:
  • Models: F:\Ai&ml\outputs\models
  • Results: F:\Ai&ml\outputs\results
  • Visualizations: F:\Ai&ml\outputs\visualization
  • SHAP Analysis: F:\Ai&ml\outputs\visualization\shap_analysis

LOADING DATA AND MODELS

✓ Initial model loaded: XGBoost
✓ Tuned model loaded

✓ Test data loaded:
  • Samples: 5000
  • Features: 20
  • Classes: ['1', '2', '3']

EVALUATING: XGBoost

📊 Overall Metrics:
  • Accuracy: 0.9598
  • Weighted F1: 0.9598
  • Macro F1: 0.9599
  • Weighted Precision: 0.9600
  • Weighted Recall: 0.9598

📋 Classification Report:
              precision  recall  f1-score    support
1                0.9637  0.9468    0.9551  1653.0000
2                0.9431  0.9627    0.9528  1688.0000
3                0.9734  0.9699    0.9716  1659.0000
accuracy         0.9598  0.9598    0.9598     0.9598
macro avg        0.9601  0.9598    0.9599  5000.0000
weighted avg     0.9600  0.9598    0.9598  5000.0000

✓ Confusion 

<Figure size 1200x800 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

<Figure size 1000x600 with 0 Axes>

...

<Figure size 2000x300 with 0 Axes>

<Figure size 2000x300 with 0 Axes>

<Figure size 2000x300 with 0 Axes>

<Figure size 2000x300 with 0 Axes>

<Figure size 2000x300 with 0 Axes>